In [1]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms, models

torch.cuda.empty_cache()

In [2]:

# Configuration
csv_file = "train.csv"
img_dir = "train/train"
test_dir = "test/test"  # Add test directory path
batch_size = 32
num_epochs = 11  # Updated number of epochs
num_classes_jenis = 2  # 0 for T-shirt, 1 for Hoodie
num_classes_warna = 5  # 0 for red, 1 for yellow, 2 for blue, 3 for black, 4 for white
learning_rate = 0.00011875811639345545  # Updated learning rate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset Class for Training
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_id = self.img_labels.iloc[idx, 0]  # Get image ID
        jenis_label = self.img_labels.iloc[idx, 1]  # Get label 'jenis'
        warna_label = self.img_labels.iloc[idx, 2]  # Get label 'warna'

        # Check if the image is in JPG or PNG format
        img_path_jpg = os.path.join(self.img_dir, f"{img_id}.jpg")
        img_path_png = os.path.join(self.img_dir, f"{img_id}.png")
        
        if os.path.exists(img_path_jpg):
            img_path = img_path_jpg
        elif os.path.exists(img_path_png):
            img_path = img_path_png
        else:
            raise FileNotFoundError(f"Image {img_id} not found in JPG or PNG format.")
        
        # Load the image
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(jenis_label), torch.tensor(warna_label)

# Custom Dataset Class for Testing
class TestDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.img_names = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))]
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_name = self.img_names[idx]
        img_path = os.path.join(self.img_dir, img_name)
        
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, img_name  # Return image and its name for reference

# Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224x224 for DenseNet
    transforms.ToTensor(),          # Convert image to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize using ImageNet statistics
])

# Create Training Dataset
df = pd.read_csv(csv_file)
class_counts = df['jenis'].value_counts().sort_index().values
class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)
sample_weights = [class_weights[label] for label in df['jenis'].values]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

dataset = CustomDataset(csv_file=csv_file, img_dir=img_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)

# Define the DenseNet model with custom classifier
class DenseNetWithCustomClassifier(nn.Module):
    def __init__(self, num_classes_jenis, num_classes_warna):
        super(DenseNetWithCustomClassifier, self).__init__()
        self.densenet = models.densenet121(pretrained=True)  # Load pre-trained DenseNet121
        num_ftrs = self.densenet.classifier.in_features  # Get input features of the classifier
        self.densenet.classifier = nn.Identity()  # Remove the original classifier

        # Define custom classifiers
        self.classifier_jenis = nn.Linear(num_ftrs, num_classes_jenis)  # For jenis
        self.classifier_warna = nn.Linear(num_ftrs, num_classes_warna)  # For warna

    def forward(self, x):
        features = self.densenet(x)  # Extract features using DenseNet
        outputs_jenis = self.classifier_jenis(features)  # Classify jenis
        outputs_warna = self.classifier_warna(features)  # Classify warna
        return outputs_jenis, outputs_warna

# Instantiate the model
model = DenseNetWithCustomClassifier(num_classes_jenis, num_classes_warna).to(device)

# Define loss functions and optimizer
criterion_jenis = nn.CrossEntropyLoss()
criterion_warna = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Use updated learning rate

# Define training function
def train(model, dataloader, criterion_jenis, criterion_warna, optimizer, num_epochs=num_epochs):
    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, jenis_labels, warna_labels in dataloader:
            images, jenis_labels, warna_labels = images.to(device), jenis_labels.to(device), warna_labels.to(device)

            optimizer.zero_grad()  # Zero the gradients

            # Forward pass
            outputs_jenis, outputs_warna = model(images)  # Get outputs from model

            # Calculate losses
            loss_jenis = criterion_jenis(outputs_jenis, jenis_labels)
            loss_warna = criterion_warna(outputs_warna, warna_labels)

            # Total loss
            loss = loss_jenis + loss_warna
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

            running_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}")

# Start training
train(model, dataloader, criterion_jenis, criterion_warna, optimizer, num_epochs)

d:\Hology\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Hology\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/11], Loss: 1.1076
Epoch [2/11], Loss: 0.2326
Epoch [3/11], Loss: 0.1064
Epoch [4/11], Loss: 0.0652
Epoch [5/11], Loss: 0.0426
Epoch [6/11], Loss: 0.0495
Epoch [7/11], Loss: 0.0261
Epoch [8/11], Loss: 0.0267
Epoch [9/11], Loss: 0.0183
Epoch [10/11], Loss: 0.0194
Epoch [11/11], Loss: 0.0204


In [4]:
def evaluate(model, test_dataloader):
    model.eval()  # Set model to evaluation mode
    correct_jenis = 0
    correct_warna = 0
    correct_both = 0
    total = 0

    with torch.no_grad():
        for images, img_names in test_dataloader:
            images = images.to(device)
            outputs_jenis, outputs_warna = model(images)

            _, predicted_jenis = torch.max(outputs_jenis, 1)
            _, predicted_warna = torch.max(outputs_warna, 1)

            total += images.size(0)

            # Assume you have the actual labels available for evaluation
            # For demonstration purposes, you can create dummy labels.
            # Replace these with actual evaluation logic as needed.
            jenis_labels = torch.randint(0, num_classes_jenis, (images.size(0),), device=device)  # Dummy labels
            warna_labels = torch.randint(0, num_classes_warna, (images.size(0),), device=device)  # Dummy labels

            correct_jenis += (predicted_jenis == jenis_labels).long().sum().item()
            correct_warna += (predicted_warna == warna_labels).long().sum().item()
            correct_both += ((predicted_jenis == jenis_labels) & (predicted_warna == warna_labels)).long().sum().item()

    accuracy_jenis = 100 * correct_jenis / total
    accuracy_warna = 100 * correct_warna / total
    accuracy_both = 100 * correct_both / total

    print(f"Accuracy (Jenis): {accuracy_jenis:.2f}%")
    print(f"Accuracy (Warna): {accuracy_warna:.2f}%")
    print(f"Accuracy (Both Correct): {accuracy_both:.2f}%")



# Create test dataset and dataloader
test_dataset = TestDataset(img_dir=test_dir, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Evaluate the model on the test set
evaluate(model, test_dataloader)


Accuracy (Jenis): 50.60%
Accuracy (Warna): 19.16%
Accuracy (Both Correct): 10.48%


In [5]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms, models
import optuna  # Import Optuna

# Configuration
csv_file = "train.csv"
img_dir = "train/train"
test_dir = "test/test"
num_classes_jenis = 2
num_classes_warna = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset Class for Training
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_id = self.img_labels.iloc[idx, 0]
        jenis_label = self.img_labels.iloc[idx, 1]
        warna_label = self.img_labels.iloc[idx, 2]

        img_path_jpg = os.path.join(self.img_dir, f"{img_id}.jpg")
        img_path_png = os.path.join(self.img_dir, f"{img_id}.png")
        
        if os.path.exists(img_path_jpg):
            img_path = img_path_jpg
        elif os.path.exists(img_path_png):
            img_path = img_path_png
        else:
            raise FileNotFoundError(f"Image {img_id} not found in JPG or PNG format.")
        
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(jenis_label), torch.tensor(warna_label)

# Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create Training Dataset
df = pd.read_csv(csv_file)
class_counts = df['jenis'].value_counts().sort_index().values
class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)
sample_weights = [class_weights[label] for label in df['jenis'].values]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

dataset = CustomDataset(csv_file=csv_file, img_dir=img_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, sampler=sampler)

# Define the DenseNet model with custom classifier
class DenseNetWithCustomClassifier(nn.Module):
    def __init__(self, num_classes_jenis, num_classes_warna):
        super(DenseNetWithCustomClassifier, self).__init__()
        self.densenet = models.densenet121(pretrained=True)
        num_ftrs = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Identity()

        self.classifier_jenis = nn.Linear(num_ftrs, num_classes_jenis)
        self.classifier_warna = nn.Linear(num_ftrs, num_classes_warna)

    def forward(self, x):
        features = self.densenet(x)
        outputs_jenis = self.classifier_jenis(features)
        outputs_warna = self.classifier_warna(features)
        return outputs_jenis, outputs_warna

# Define training function with Optuna
def train_model(trial):
    # Hyperparameter tuning
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    num_epochs = trial.suggest_int('num_epochs', 5, 20)
    batch_size = trial.suggest_int('batch_size', 16, 64)
    
    # Adjust dataloader for the current trial's batch size
    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)

    model = DenseNetWithCustomClassifier(num_classes_jenis, num_classes_warna).to(device)
    criterion_jenis = nn.CrossEntropyLoss()
    criterion_warna = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, jenis_labels, warna_labels in dataloader:
            images, jenis_labels, warna_labels = images.to(device), jenis_labels.to(device), warna_labels.to(device)

            optimizer.zero_grad()
            outputs_jenis, outputs_warna = model(images)
            loss_jenis = criterion_jenis(outputs_jenis, jenis_labels)
            loss_warna = criterion_warna(outputs_warna, warna_labels)
            loss = loss_jenis + loss_warna
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Report the loss at the end of the epoch
        avg_loss = running_loss / len(dataloader)
        trial.report(avg_loss, epoch)

        # Stop early if the loss is not improving
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return avg_loss  # Return the average loss

# Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(train_model, n_trials=20)

# Output best hyperparameters
print("Best hyperparameters: ", study.best_params)
print("Best loss: ", study.best_value)


d:\Hology\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-10-26 15:12:01,284] A new study created in memory with name: no-name-bd7e089e-943d-4ddf-87ea-d0a4d3d883f6
C:\Users\lmaos\AppData\Local\Temp\ipykernel_24672\3214744971.py:88: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
d:\Hology\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Hology\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight

KeyboardInterrupt: 